<a href="https://colab.research.google.com/github/shivaprajapati34390-netizen/ML-project/blob/main/online_payments_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# I will start this task by importing the necessary Python libraries and the dataset we need for this task:

In [67]:
import pandas as pd
import numpy as np


In [68]:
# Now, let’s have a look at whether this dataset has any null values or not:

In [69]:
data = pd.read_csv(
    "credit_card.csv",
    sep=",",
    header=None,
    skiprows=1,      # <-- drop the malformed first line
    engine="python"
)

print(data.head())
print(data.shape)


    0         1          2            3    4    5            6           7   \
0  132  CASH_OUT  170723.79  C1267433600  0.0  0.0   C405438084   754766.29   
1  132  CASH_OUT  134059.98   C304349420  0.0  0.0  C1045479759  1593953.06   
2  132  CASH_OUT  219135.71   C924424830  0.0  0.0  C1726301214   956825.09   
3  132  CASH_OUT  168670.60   C930877822  0.0  0.0  C1680525470  1523401.63   
4  132  CASH_OUT   78693.03   C543426705  0.0  0.0  C1808508403   111498.46   

           8   9    10  
0   925490.09   0  0.0  
1  1728013.04   0  0.0  
2  1175960.80   0  0.0  
3  1692072.23   0  0.0  
4   190191.49   0  0.0  
(80663, 11)


In [70]:
print(data.isnull())

          0      1      2      3      4      5      6      7      8      9   \
0      False  False  False  False  False  False  False  False  False  False   
1      False  False  False  False  False  False  False  False  False  False   
2      False  False  False  False  False  False  False  False  False  False   
3      False  False  False  False  False  False  False  False  False  False   
4      False  False  False  False  False  False  False  False  False  False   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
80658  False  False  False  False  False  False  False  False  False  False   
80659  False  False  False  False  False  False  False  False  False  False   
80660  False  False  False  False  False  False  False  False  False  False   
80661  False  False  False  False  False  False  False  False  False  False   
80662  False  False  False  False  False  False  False  False  False  False   

          10  
0      False  
1      False  
2     

In [71]:
print(data.isnull().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    1
dtype: int64


In [72]:
data.head()
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80663 entries, 0 to 80662
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       80663 non-null  int64  
 1   1       80663 non-null  object 
 2   2       80663 non-null  float64
 3   3       80663 non-null  object 
 4   4       80663 non-null  float64
 5   5       80663 non-null  float64
 6   6       80663 non-null  object 
 7   7       80663 non-null  float64
 8   8       80663 non-null  float64
 9   9       80663 non-null  int64  
 10  10      80662 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 6.8+ MB


In [73]:
data.columns = [
    'step',
    'type',
    'amount',
    'nameOrig',
    'oldbalanceOrg',
    'newbalanceOrig',
    'nameDest',
    'oldbalanceDest',
    'newbalanceDest',
    'isFraud',
    'isFlaggedFraud'
]


In [74]:
data['type'].value_counts()


,count
type,
CASH_OUT,30625
PAYMENT,24613
CASH_IN,18307
TRANSFER,6738
DEBIT,380


In [75]:
data.shape

(80663, 11)

In [76]:
# this data does not have null values

In [77]:
print(data.columns)


Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')


In [78]:
type_counts = data['type'].value_counts()


In [79]:
data['type'].value_counts()
transactions=type_counts.index
quantity=type_counts.values

import plotly.express as px
figure = px.pie(data,
                values=quantity,
                names=transactions,hole=0.5,
                title="Distribution of Transaction type")
figure.show()

In [80]:
# Now let’s have a look at the correlation between the features of the data with the isFraud column:


In [81]:
# checking correletion
correlation = data. select_dtypes(include ='number').corr()
print(correlation["isFraud"].sort_values(ascending=False))

isFraud           1.000000
amount            0.109502
oldbalanceOrg     0.006282
newbalanceDest    0.001400
step             -0.001557
newbalanceOrig   -0.004200
oldbalanceDest   -0.004605
isFlaggedFraud         NaN
Name: isFraud, dtype: float64


In [82]:
# now lets transform the categorical features into numerical features: here i will also transform the value of isFraud column into No Fraud and Fraud labels to have a better understanding of the output:

In [83]:
data['type'] = data ['type'].map({'CASH_OUT': 1, 'PAYMENT': 2, 'CASH_IN': 3, 'TRANSFER': 4, 'DEBIT': 5})
data['isFraud'] = data['isFraud'].map({0: "No Fraud", 1: "Fraud"})
print(data.head())

   step  type     amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0   132     1  170723.79  C1267433600            0.0             0.0   
1   132     1  134059.98   C304349420            0.0             0.0   
2   132     1  219135.71   C924424830            0.0             0.0   
3   132     1  168670.60   C930877822            0.0             0.0   
4   132     1   78693.03   C543426705            0.0             0.0   

      nameDest  oldbalanceDest  newbalanceDest   isFraud  isFlaggedFraud  
0   C405438084       754766.29       925490.09  No Fraud             0.0  
1  C1045479759      1593953.06      1728013.04  No Fraud             0.0  
2  C1726301214       956825.09      1175960.80  No Fraud             0.0  
3  C1680525470      1523401.63      1692072.23  No Fraud             0.0  
4  C1808508403       111498.46       190191.49  No Fraud             0.0  


In [84]:
# now lets train the classification model to classify whether a transactions is fraud or not:

In [85]:
from sklearn.model_selection import train_test_split
x = np.array(data[["type", "amount", "oldbalanceOrg", "newbalanceOrig"]])
y = np.array(data[["isFraud"]])

In [86]:
# now lets train the online payment fraud detection model:

In [87]:
# train a machine learning model
from sklearn.tree import DecisionTreeClassifier
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=42)
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)

DecisionTreeClassifier()

In [88]:
print(model.score(xtest, ytest))

0.9998760381802405


In [89]:
# now lets classifier whether the transaction is fraud or not:


In [90]:
# prediction
feature=features = np.array([[4, 9000.60, 9000.60, 0.0]])
print(model.predict(features))

['No Fraud']


In [91]:
data['isFraud'].value_counts()


,count
isFraud,
No Fraud,80639
Fraud,24
